In [1]:
%run MusicData.ipynb

# from music21 import *
# import numpy as np
# import torch
# import pretty_midi
# import requests, requests_html
# import os
# import sys
# import pickle
# import time
# import random
# import re

### data pre-pocessing

In [2]:
def file_spliter(input_file, output_folder, regularization = True):
    with open(input_file, 'r') as f:
        temp_list = list()
        counter = 1
        for line in f:
            if line == '\n':
                continue
            if line.startswith('X:'):
                if len(temp_list) > 0:
                    # print(temp_list)
                    origin = ([line for line in temp_list if line.startswith('O:')] + ['None'])[0]
                    if regularization:
                        temp_list, is_throw = regular_english(temp_list)             
                        if is_throw is True:
                            temp_list = list()
                            continue
                    with open(f"{output_folder}/{counter}.abc", 'w') as out:
                        print(temp_list)
                        out.writelines(temp_list)
                    counter += 1
                    temp_list = list()
            temp_list.append(line)
    print(counter)

In [3]:
def regular(input_list):
    output_list = input_list.copy()
    tempo_token = None
    culture_token = None
    meter_token = None
    throw_token = False
    for i, line in enumerate(input_list):
        if line.startswith('Q:'):
            tempo_token = True
            output_list[i] = 'Q:1/4=120\n'
        if line.startswith('X:'): #ONLY FOR IRISH
            output_list[i] = f'X: {i}\n'
        if line.startswith('O:') and 'england' in line.lower():
            culture_token = True
            output_list[i] = 'O:English\n'
        if 'V:' in line:
            throw_token = True
        if line.startswith('K:') and 'maj' in line.lower():
            output_list[i] = output_list[i][:-4]+'\n'
        if line.startswith('L:') and '32' in line:
            output_list[i] = output_list[i][:-3] + '16\n'
    if tempo_token == None:
        output_list.insert(2,'Q:120\n')
    if culture_token == None:
        output_list.insert(2,'O:Chinese\n')
        # output_list.insert(2,'O:Irish\n')
        # throw_token = True
    # for english version, remove TBL line
    output_list = [line for line in putput_list if line[:2]!='%%']
    return output_list, throw_token

In [4]:
def regular_english(input_list):
    output_list = input_list.copy()
    tempo_token = None
    culture_token = None
    meter_token = None
    throw_token = False
    for i, line in enumerate(input_list):
        if line.startswith('Q:'):
            tempo_token = True
            output_list[i] = 'Q:1/4=120\n'
        if line.startswith('X:'): #ONLY FOR IRISH
            output_list[i] = f'X: {i}\n'
        if line.startswith('O:') and 'england' in line.lower():
            culture_token = True
            output_list[i] = 'O:English\n'
        if line.startswith('V:'):
            throw_token = True
        if "[M:" in line: # 变速
            throw_token = True
#         if line.startswith('K:') and 'maj' in line.lower():
#             output_list[i] = output_list[i][:-4]+'\n'
        if line.startswith('L:') and '32' in line:
            output_list[i] = output_list[i][:-3] + '16\n'
        if 'T' in line and line[1] != ':':
            output_list[i].strip('T')
        
    if tempo_token == None:
        output_list.insert(2,'Q:120\n')
    if culture_token == None:
        # output_list.insert(2,'O:Chinese\n')
        # output_list.insert(2,'O:Irish\n')
        throw_token = True
    # for english version, remove TBL line
    output_list = [line for line in output_list if line[:2]!='%%']
    output_list = [line for line in output_list if line[:2]!='N:']
    output_list = [line for line in output_list if line[:2]!='W:']
    return output_list, throw_token

In [5]:
def regular_chinese(input_list):
    output_list = input_list.copy()
    tempo_token = None
    culture_token = None
    meter_token = None
    throw_token = False
    for i, line in enumerate(input_list):
        if line.startswith('Q:'):
            tempo_token = True
            output_list[i] = 'Q:1/4=120\n'
        if line.startswith('O:'):
            culture_token = True
            output_list[i] = 'O:Chinese\n'
        if line.startswith('V:'):
            throw_token = True
        if line.startswith('L:') and '32' in line:
            output_list[i] = output_list[i][:-3] + '16\n'
    if tempo_token == None:
        output_list.insert(2,'Q:120\n')
    if culture_token == None:
        output_list.insert(2,'O:Chinese\n')
    return output_list, throw_token

In [37]:
# input_file = "/gpfsnyu/home/yz6492/multimodal/data/abc/tunebook.abc"
# output_folder = "/gpfsnyu/home/yz6492/multimodal/data/abc/english/"
# file_spliter(input_file, output_folder)

['T:G E. WHG.029\n', 'T:?\n', 'A:Bampton, Oxfordshire\n', 'F:http://richardrobinson.tunebook.org.uk/tune/8518\n', 'H:donated by Mrs W.R.Kettlewell to VWML,Apr 1927.\n', 'L:1/8\n', 'M:6/8\n', 'O:English\n', 'Q:1/4=120\n', 'R:Jig\n', 'S:W.H.Giles MS,Bampton,Oxfordshire,1839p27\n', 'Z:PJHeadford - VMP 2009\n', 'K:C\n', 'c/d/|e>fe d>ge|c>Bc dBc {A}|G>AF E>Gc|c>de d2c/d/|\n', 'e>fe dge|cBc d>Bc {A}|G>AF E<Gc|d>AB c2||\n', 'e|e>cd e>af|ecd e>cd|dcB dge|dcA {A} G2G/c/|\n', 'ced ecd|ege f<de|f>ga g>ec|d>AB c/B/c/A/c/d/|\n', 'G>AF E<G c/d/|e>ge f<de|f>ga g>ec|d>AB c2|]\n']
['X: 0\n', 'T:Birthday of the Little Doctor,The. Car.09\n', 'T:?\n', 'A:Carlisle\n', 'F:http://richardrobinson.tunebook.org.uk/tune/7475\n', 'L:1/8\n', 'M:6/8\n', 'O:English\n', 'Q:1/4=120\n', 'R:.Jig\n', "S:CarlisleMS,Anon.c1812.AGG's transcription\n", 'Z:vmp.Chris Partington\n', 'K:Eb\n', 'B/A/|GBe GBe|c3 f2e|dcB Bcd|e3 g2B/A/|!\n', 'GBe GBe|c3 f2e|dcB Bcd|e2z z2||!\n', 'b/a/|g2b f2b|e2b d2b|ggb ffb|{b}aga b2b|!\n', 'e2e e2

['X: 0\n', "T:Aldridge's Dance\n", 'Q:120\n', 'A:Wyresdale, Lancashire\n', 'B:H.S.J. Jackson, 1823\n', 'F:http://richardrobinson.tunebook.org.uk/tune/3670\n', 'L:1/8\n', 'M:6/8\n', 'O:English\n', 'Z:Richard Robinson <URL:http://www.qualmograph.org.uk/contact.html>\n', 'K:G\n', 'd/c/ |\\\n', 'B2G GAG | B2G G2G | A2B c2B | A2G FED |\\\n', 'G2A B2c | d2d d2B | A2B c2B | A2B Hc2 d/c/ |\n', 'B2G GAG | B2G GAG | A2B c2B | A2G FED |\\\n', 'G2A B2c | d2d Hd2 e/f/ | g2d dec | B2G G2 ::\n']
["T:Aldridge's Dance. HSJJ.060\n", 'A:Lancashire\n', 'F:http://richardrobinson.tunebook.org.uk/tune/9145\n', 'H:1823\n', 'L:1/8\n', 'M:6/8\n', 'O:English\n', 'Q:1/4=120\n', 'R:.Jig\n', 'S:HSJ Jackson,Wyresdale,Lancs.1823\n', 'Z:vmp.Chris Partington.\n', 'K:G\n', 'd/c/|B2G GAG|B2G G2G|A2Bc2B|A2G FED|!\n', 'G2AB2c|d2dd2B|A2Bc2B|A2BHc2d/c/|!\n', 'B2G GAG|B2G GAG|A2Bc2B|A2G FED|!\n', 'G2AB2c|d2dHd2e/f/|g2d dec|B2GG2:|\n']
['X: 0\n', 'T:Aldriges Allemande. WM.030\n', "T:Aldridge's Allemande,aka. WM.030\n", 'A:Kent

['X: 0\n', "T:Arne's Jig. JBi.42\n", "T:Aron's Maggot,aka. JBi.42\n", 'A:Yorkshire\n', 'C:Thomas Arne? 1710-1778\n', 'F:http://richardrobinson.tunebook.org.uk/tune/7018\n', 'L:1/8\n', 'M:C|\n', 'O:English\n', 'Q:1/4=120\n', 'R:.Reel\n', 'S:James Biggins MS, Leeds, 1779.\n', 'Z:vmp.Nick Barber & Chris Partington\n', 'K:G\n', 'G2BG (B/c/d) BG|F2AF (A/B/c) AF|\\\n', 'G2BG (B/c/d) gB|cAdB|G/G/GG2:|!\n', '|:d|g2bg afed|e2f2gagd|g2bg afed|(e/f/g) (f/g/a)Tg3B|!\n', 'ceAc BdGB|Aagf edcB|ceAc BdGB|(A/B/c) AFG4:|\n']
['X: 0\n', 'T:Arrow Waltz,The. CJF.204\n', 'A:East Yorkshire\n', 'B:Kidson Coll.Mitchell Lib.M1805\n', 'F:http://richardrobinson.tunebook.org.uk/tune/8245\n', 'L:1/16\n', 'M:3/8\n', 'O:English\n', 'Q:1/4=120\n', 'R:Waltz\n', 'S:C.J.Fox MS, 1829/33, Beverley,E.Yorks.\n', 'Z:vmp.P J Headford 2000 & 2008\n', 'K:C\n', "G2|c2{d'}c'2G2|A2{^g}a2E2|F2D2B2|c2e2 G2|\\\n", "c2{b}c'2 G2|A2{^g}a2 E2|(FE).F.G.A.B|c4:|!\n", '|:e2|" cres:".f2.d\'2f2|.e2.c\'2 e2|\\\n', "d2G2g2|e2c2 e2|.f2.d'2 f2|e2c

['X: 0\n', 'T:Lango Lee. RH.190\n', 'T:Banks of the Dee,The. RH.190\n', 'A:Temple Sowerby,Cumbria\n', 'F:http://richardrobinson.tunebook.org.uk/tune/8761\n', 'L:1/8\n', 'M:6/8\n', 'O:English\n', 'Q:1/4=120\n', 'R:Jig\n', "S:Rev.R.Harrison's MS,c1815,Cumbria\n", 'Z:vmp.Simon Wilson. Review PJH, 2008.\n', 'K:G\n', 'D|D>EF G>AB|c>ed cBA|B>GE D>EG|B>cA G2D|\\\n', 'D>EF G>AB|c>ed cBA|B>GE D>EG|B>cA G2:|!\n', '|:c|B2c d3|e>ce d>BG|B>cd e>fg|G>AG F>ED|\\\n', 'c>ec B>dB|A>GA BGE|G>FE Ddc|B>cA G2:|\n']
["T:James McKenney's Hornpipe. WI.004\n", 'T:Bantry Bay,aka. WI.004\n', 'A:Lake District\n', 'F:http://richardrobinson.tunebook.org.uk/tune/9494\n', 'L:1/16\n', 'M:2/4\n', 'O:English\n', 'Q:1/4=120\n', 'R:.Hornipe\n', "S:Wm Irwin, 1838 MS, AGG's Transcription\n", 'Z:vmp.Chris Partington.2005\n', 'K:G\n', 'dc|BGAG EGDE|GG2G G3B|AGAB cABG|E2A2 A2B2|!\n', 'c2ec B2dB|ABAG E2dc|BABG EGDE|G2G2 G2:|!\n', '|:Bc|dBGB dBGA|B2d2 d2f2|gfge dege|e2g2 g2(3def|!\n', 'gfgf edcB|gfgf edcB|cABG EGDE|G2G2 G2:|\n']


['X: 0\n', 'T:Bell-sids-Minuet. JaW.091\n', 'T:Belsize Minuet?aka. JaW.091\n', 'A:Wyresdale,Lancashire\n', 'F:http://richardrobinson.tunebook.org.uk/tune/10660\n', 'L:1/4\n', 'M:3/4\n', 'O:English\n', 'Q:1/4=120\n', 'R:.Minuet\n', 'S:James Winder Ms, Lancashire, 1835-41\n', 'Z:vmp.Chris Partington, Aug 2004\n', 'K:G\n', 'G2A|B2d/e/|dcB|A2B|ce/c/A/c/|Bd/B/G/B/|!\n', 'c/d/ e/c/A/c/|B/c/ d/B/G/B/|AGF|G3|!\n', '|dgf|gab|agf|e2f|gb/g/e/g/|!\n', 'f"_g in MS"a/f/d/f/|g/a/ b/g/e/g/|f/g/ a/f/d/f/|ed" nat. in MS"^c|d3|!\n', 'dg2|c/d/ e/c/A/c/|B/c/ d/B/G/B/|A"="D2|!\n', 'G2"+"A|B/A/ B/c/d/B/|G2"+"A|B/A/ B/c/d/B/|gBA|G3||\n']
['X: 0\n', 'T:Beneath this Consecrated roof. WHG.060\n', 'A:Bampton, Oxfordshire\n', 'F:http://richardrobinson.tunebook.org.uk/tune/8551\n', 'H:donated by Mrs W.R.Kettlewell to VWML,Apr 1927.\n', 'L:1/4\n', 'M:2/2\n', 'O:English\n', 'Q:1/4=120\n', 'R:hymn\n', 'S:W.H.Giles MS,Bampton,Oxfordshire,1839p63\n', 'Z:PJHeadford - VMP 2009\n', 'K:G\n', 'd2 cB|A2 B2|cd Bd/c/|(B2 A2)|\\

...

['X: 0\n', 'T:Waltz. CJF.100\n', 'A:East Yorkshire\n', 'B:Kidson Coll.Mitchell Lib.M1805\n', 'F:http://richardrobinson.tunebook.org.uk/tune/8141\n', 'L:1/16\n', 'M:2/4\n', 'O:English\n', 'Q:1/4=120\n', 'R:schot\n', 'S:C.J.Fox MS, 1829/33, Beverley,E.Yorks.\n', 'Z:vmp.P J Headford 2000 & 2008\n', 'K:G\n', "Bc|d2g2 fgaf|g2b2 d2^d2|e2c'b abag|fgfe dcBc|\\\n", "d2g2 fgaf|g2b2 d2^d2|ecc'a g2f2|Hg4 dcBc:|!\n", 'dgfa gbgd|dcBc A3B|dfag fedc|B2G2 GBdg|\\\n', 'egfg agfg|dgfg agfg|ec\'e\'d\' c\'d\'c\'b|agfe " D.C."dcBc|]\n']
['X: 0\n', 'T:Waltz. CJF.101\n', 'A:East Yorkshire\n', 'B:Kidson Coll.Mitchell Lib.M1805\n', 'C:"From J.Johnson.C J Fox 1829"\n', 'F:http://richardrobinson.tunebook.org.uk/tune/8142\n', 'L:1/16\n', 'M:3/8\n', 'O:English\n', 'Q:1/4=120\n', 'R:waltz\n', 'S:C.J.Fox MS, 1829/33, Beverley,E.Yorks.\n', 'Z:vmp.P J Headford 2000 & 2008\n', 'K:G\n', "d>c|B>dg>fa>g|b>gdzB2|((3cde) ((3fga) b>d'|c'>ba>gf>e|\\\n", 'd>gf>ga>g|b>gdzB2|(3cec (3AcA D>F|G4:|!\n', '|:G2|FAGABc|B>d d3g|gfedcB|B

['X: 0\n', 'T:Waterloo Bridge. GS.069\n', 'A:Leeds\n', 'F:http://richardrobinson.tunebook.org.uk/tune/10251\n', 'H:1831\n', 'L:1/8\n', 'M:6/8\n', 'O:English\n', 'Q:1/4=120\n', 'R:.Quadrille\n', 'S:George Spencer m/s, Leeds,1831\n', 'Z:vmp.Cherri Graebe\n', 'K:C major\n', '"_see note"C | E2 E FDF | Ged cBA | G2 E G2 E | GFE DEF |!\n', 'E2 E FEF | Ged cBA | G2EG2E | FDB, C2 ||!\n', 'G | c2c cBc | d2 G GFG | d2 d dcd | e2 c c2G |!\n', 'EGc cBc | FAd dcd |  egf edc | BAG FE"^Da Capo"D |]\n']
['X: 0\n', 'T:Waterloo. CJF.155\n', 'A:East Yorkshire\n', 'B:Kidson Coll.Mitchell Lib.M1805\n', 'F:http://richardrobinson.tunebook.org.uk/tune/8196\n', 'L:1/8\n', 'M:6/8\n', 'O:English\n', 'Q:1/4=120\n', 'R:Quadrille\n', 'S:C.J.Fox MS, 1829/33, Beverley,E.Yorks.\n', 'Z:vmp.P J Headford 2000 & 2008\n', 'K:F\n', 'c|c2d A2B|{d}c2c caf|ebg ecG|BAA A2B|\\\n', 'c2d A2B|{d}c2c caf|dBG EcE|G3F"*"zH||!\n', 'F|E2G c2B|A2c F2c|^cdA ^cBG|F3 E2F|\\\n', 'E2G c2B|A2c Fag|geg fd=B|d3 c2||!\n', 'A|d2f afd|{d}^c2c c2b|b

['X: 0\n', "T:What's more pleasing to the Eye. CJF.206\n", 'A:East Yorkshire\n', 'B:Kidson Coll.Mitchell Lib.M1805\n', 'F:http://richardrobinson.tunebook.org.uk/tune/8247\n', 'L:1/8\n', 'M:2/4\n', 'O:English\n', 'Q:1/4=120\n', 'R:Air\n', 'S:C.J.Fox MS, 1829/33, Beverley,E.Yorks.\n', 'Z:vmp.P J Headford 2000 & 2008\n', 'K:C\n', 'e2e2|d2d2|c2(ec)|G2z2|\\\n', 'f2e2|d2c2|B2c2|(d3^d)|!\n', 'e2e2|d2d2|c2(ec)|G2z2|\\\n', 'A2A2|G2c2|(ed) (cB)|c4H||!\n', 'B2G2|d3c|B2G2|D2z2|\\\n', 'c2B2|e2d2|^F2G2|A2z2|!\n', 'A2A2|B2B2|c2d2|e2z2|\\\n', 'd2c2|(Bg) (ec)|(B2"tr"A3/2) G/|" D.C."G4H|]\n']
["T:When a Man's a Little Bit Poorly. RH.511\n", 'T:Bab At The Bowster,aka. RH.511\n', 'T:Country Bumpkin,aka. RH.511\n', 'A:Temple Sowerby,Cumbria\n', 'F:http://richardrobinson.tunebook.org.uk/tune/9082\n', 'L:1/8\n', 'M:6/8\n', 'O:English\n', 'Q:1/4=120\n', 'R:air\n', "S:Rev.R.Harrison's MS,c1815,Cumbria\n", 'Z:vmp.Simon Wilson. Review PJH, 2008.\n', 'K:G\n', 'c2c B2B|ABG FED|(G2B) .B2z|(G2B) .B2z|!\n', 'c2c B>BB

['X: 0\n', 'T:the Windermere Regatta\n', 'Q:120\n', 'A:North-West\n', 'F:http://richardrobinson.tunebook.org.uk/tune/1281\n', 'M:4/4\n', 'O:English\n', 'R:Hornpipe\n', 'Z:Richard Robinson <URL:http://www.qualmograph.org.uk/contact.html>\n', 'K:Bb\n', 'F|\\\n', 'FEDB dfdB| cBAc egec| \\\n', 'Bfdf egdf| cfAc BABG| \\\n', 'FEDB dfdB| cBAc egec|\n', 'Bfdf cabe| d2B2 B2z::\\\n', 'g|\\\n', 'fede fbfd| fgdf c2Ac|\n', 'eeec defg| BAcA F2FE| \\\n', 'dB,DF dfff| EDEC eggg| \\\n', 'Bfdf cabe| d2B2 B2:|\n']
['X: 0\n', 'T:Windermere Regatta. WI.035\n', 'A:Lake District\n', 'C:"A Hornpipe"\n', 'F:http://richardrobinson.tunebook.org.uk/tune/9525\n', 'L:1/8\n', 'M:C|\n', 'O:English\n', 'Q:1/4=120\n', 'R:.Hornpipe\n', "S:Wm Irwin, 1838 MS, AGG's Transcription\n", 'Z:vmp.Chris Partington.2005\n', 'K:Bb\n', '"qu"F2|FEDB dfdB|cBAc egec|Bfdf egdf|cfAc BABG|!\n', 'FEDB dfdB|cBAc egec|Bfdf cabe|d2B2 B2:|!\n', '|:"qu"f2|fede fbfd|fgdf c2Fd|eeeB defg|BAcA F2FE|!\n', 'DB,DF dfff|EDEC eggg|Bfdf cabe|d2B2 B2:|\n'

['X: 0\n', "T:the Young Man's Fancy\n", 'Q:120\n', 'A:Wyresdale, Lancashire\n', 'B:Winder\n', 'F:http://richardrobinson.tunebook.org.uk/tune/1547\n', 'L:1/8\n', 'M:4/4\n', 'O:English\n', 'Z:Richard Robinson <URL:http://www.qualmograph.org.uk/contact.html>\n', 'K:E min\n', 'BA |\\\n', 'GFGE g2fe | dBAG c2BA | \\\n', 'GFGE DEFD | d2A2 A2BA | \\\n', 'GFGE g2fe | dBAG c2BA |\n', 'GFGE DcBA | G2E2 E2 :: \\\n', 'B2 |\\\n', 'e2e2 efgf | e2B2 B2Bc | \\\n', 'd2d2 defe | d2A2 A2gf |\n', 'edef g2fe | dBAG c2BA | \\\n', 'GFGE DcBA | G2E2 E2 :: \\\n', 'GA | \\\n', 'BGEG BGEG | BAGF E2FG |\n', 'AFDF AFDF | AGFE D2GA | \\\n', 'BGEG BGEG | BGFE g2fe | \\\n', 'dBAG AcBA | G2E2 E2 :|\n']
['X: 0\n', 'T:May Moon,The. JMT.040\n', 'T:Young May Moon,aka. JMT.040\n', 'A:Northumbria\n', 'F:http://richardrobinson.tunebook.org.uk/tune/10037\n', 'L:1/8\n', 'M:6/8\n', 'O:English\n', 'Q:1/4=120\n', 'R:.Jig\n', 'S:J,Moore,Tyneside,1841.(29a)\n', 'Z:vmp.J.Adam\n', 'K:D\n', 'A|d2A AFA|d2A AFA|d2d ede|f2d def|!\n', 'g2

### convert abc sequence to midi and matrix

In [59]:
# s = converter.parse(f"/gpfsnyu/home/yz6492/multimodal/data/abc/chinese/16.abc")
# s.show('midi')

In [2]:
data_object = MusicData(f"/gpfsnyu/home/yz6492/multimodal/data/abc/irish/12909.abc", culture = 'Irish')

Error in Matrix. Fixed? False


In [6]:
file_list = list()

temp_list = os.listdir("/gpfsnyu/home/yz6492/multimodal/data/abc/irish")
counter = 0
for i, file in enumerate(temp_list):
    data_object = MusicData(f"/gpfsnyu/home/yz6492/multimodal/data/abc/irish/{file}", culture = 'Irish')
    if data_object.valid:
        file_list.append(data_object)
        counter += 1
    if counter >= 10000:
        break

print(len(file_list))

temp_list = os.listdir("/gpfsnyu/home/yz6492/multimodal/data/abc/chinese")
counter = 0
for i, file in enumerate(temp_list):
    data_object = MusicData(f"/gpfsnyu/home/yz6492/multimodal/data/abc/chinese/{file}", culture = 'Chinese')
    if data_object.valid:
        file_list.append(data_object)
        counter += 1
#     if counter >= 100:
#         break
        
print(len(file_list))

temp_list = os.listdir("/gpfsnyu/home/yz6492/multimodal/data/abc/english")
counter = 0
for i, file in enumerate(temp_list):
#    try:
    data_object = MusicData(f"/gpfsnyu/home/yz6492/multimodal/data/abc/english/{file}", culture = 'English')
    if data_object.valid:
        file_list.append(data_object)
        counter += 1
#     except Exception as e:
#         print(file)
#         with open(f"/gpfsnyu/home/yz6492/multimodal/data/abc/english/{file}", 'r') as f:
#             print(f.readlines())
#     if counter >= 100:
#         break

print(len(file_list))

with open("/gpfsnyu/scratch/yz6492/multimodal/data/MusicData_full.pkl",'wb') as f:
    pickle.dump(file_list, f)

Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in M

Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in M

Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in parsing: id - stream
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in M

Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in parsing: id - stream
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in M

abcFormat: WARNING: Could not get pitch information from note:  W, assuming C
abcFormat: WARNING: Could not get pitch information from note:  W, assuming C


...

Error in Matrix. Fixed? True
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? True
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False


abcFormat: WARNING: Could not get pitch information from note:  W, assuming C
abcFormat: WARNING: Could not get pitch information from note:  W, assuming C
abcFormat: WARNING: Could not get pitch information from note:  W, assuming C
abcFormat: WARNING: Could not get pitch information from note:  W, assuming C
abcFormat: WARNING: Could not get pitch information from note:  W, assuming C
abcFormat: WARNING: Could not get pitch information from note:  W, assuming C
abcFormat: WARNING: Could not get pitch information from note:  W, assuming C
abcFormat: WARNING: Could not get pitch information from note:  W, assuming C
abcFormat: WARNING: Could not get pitch information from note:  W, assuming C
abcFormat: WARNING: Could not get pitch information from note:  W, assuming C


Error in Matrix. Fixed? True
Error in parsing: id - meter
Error in Matrix. Fixed? False
Error in parsing: id - stream
Error in parsing: id - meter
Error in parsing: id - stream
Error in Matrix. Fixed? True
Error in Matrix. Fixed? True
Error in Matrix. Fixed? True
Error in Matrix. Fixed? False
Error in Matrix. Fixed? True
Error in parsing: id - stream
Error in Matrix. Fixed? True
Error in Matrix. Fixed? True
Error in Matrix. Fixed? True
Error in Matrix. Fixed? True
Error in Matrix. Fixed? True
Error in parsing: id - stream
Error in Matrix. Fixed? True
Error in Matrix. Fixed? False
Error in Matrix. Fixed? True
Error in parsing: id - stream
Error in Matrix. Fixed? True
Error in Matrix. Fixed? True
Error in parsing: id - stream
Error in Matrix. Fixed? False
Error in Matrix. Fixed? True
Error in Matrix. Fixed? False
Error in Matrix. Fixed? True
Error in Matrix. Fixed? False
Error in Matrix. Fixed? True
Error in Matrix. Fixed? False
Error in parsing: id - stream
Error in parsing: id - stream

abcFormat: WARNING: Could not get pitch information from note:  W, assuming C
abcFormat: WARNING: Could not get pitch information from note:  W, assuming C
abcFormat: WARNING: Could not get pitch information from note:  W, assuming C
abcFormat: WARNING: Could not get pitch information from note:  W, assuming C
abcFormat: WARNING: Could not get pitch information from note:  W, assuming C
abcFormat: WARNING: Could not get pitch information from note:  W, assuming C
abcFormat: WARNING: Could not get pitch information from note:  W, assuming C
abcFormat: WARNING: Could not get pitch information from note:  W, assuming C


Error in Matrix. Fixed? False
Error in Matrix. Fixed? True
Error in Matrix. Fixed? True
Error in parsing: id - stream
Error in Matrix. Fixed? True
Error in Matrix. Fixed? False
Error in Matrix. Fixed? True
Error in parsing: id - stream
Error in parsing: id - stream
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in parsing: id - stream
Error in parsing: id - stream
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in parsing: id - stream
Error in Matrix. Fixed? False
Error in Matrix. Fixed? True
Error in parsing: id - stream
Error in Matrix. Fixed? True
Error in Matrix. Fixed? False
Error in Matrix. Fixed? False
Error in parsing: id - meter
Error in parsing: id - stream
Error in parsing: id - stream
Error in parsing: id - stream
Error in Matrix. Fixed? False
Error in Matrix. Fixed? True
Error in parsing: id - meter
Error in Matrix. Fixed? True
Error in parsing: id - meter
Error in Matrix. Fixe

In [18]:
# temp_list = os.listdir("/gpfsnyu/home/yz6492/multimodal/data/abc/chinese")
# for i, file in enumerate(temp_list[:2000]):

SyntaxError: unexpected EOF while parsing (<ipython-input-18-294d67d00e6b>, line 2)

In [4]:
with open("/gpfsnyu/scratch/yz6492/multimodal/data/MusicData_full.pkl", 'rb') as f:
    counter = 0
    filter_list = list()
    print('in')
    data = pickle.load(f)
    for line in data:
        if line.meter[0] in ['3','4'] and ('major' in line.key or 'minor' in line.key) and line.culture != 'Irish':
            print(line.culture, line.key, line.meter, line.torch_matrix.shape)
            filter_list.append(line)
        counter += 1
    with open("/gpfsnyu/scratch/yz6492/multimodal/data/MusicData_no_Irish.pkl", 'wb') as g:
        pickle.dump(filter_list, g)
    print(counter)

in
Chinese B- major 3/4 torch.Size([228, 130])
Chinese G major 4/4 torch.Size([192, 130])
Chinese D major 4/4 torch.Size([96, 130])
Chinese A major 3/4 torch.Size([379, 130])
Chinese A- major 3/2 torch.Size([856, 130])
Chinese D major 4/4 torch.Size([80, 130])
Chinese D major 4/4 torch.Size([128, 130])
Chinese D major 4/4 torch.Size([192, 130])
Chinese D major 4/4 torch.Size([96, 130])
Chinese D major 3/4 torch.Size([239, 130])
Chinese E- major 4/4 torch.Size([192, 130])
Chinese D major 3/4 torch.Size([96, 130])
Chinese F major 3/8 torch.Size([114, 130])
Chinese A major 3/4 torch.Size([76, 130])
Chinese A major 3/4 torch.Size([152, 130])
Chinese C major 4/4 torch.Size([304, 130])
Chinese G major 3/2 torch.Size([80, 130])
Chinese C major 3/4 torch.Size([108, 130])
Chinese A major 4/4 torch.Size([184, 130])
Chinese D major 3/4 torch.Size([136, 130])
Chinese B- major 4/4 torch.Size([552, 130])
Chinese B- major 3/8 torch.Size([66, 130])
Chinese C major 3/4 torch.Size([96, 130])
Chinese F m

English D major 4/4 torch.Size([250, 130])
English D major 3/4 torch.Size([192, 130])
English D major 4/4 torch.Size([752, 130])
English G major 3/2 torch.Size([288, 130])
English G major 4/4 torch.Size([520, 130])
English D major 4/4 torch.Size([256, 130])
English D major 4/4 torch.Size([254, 130])
English G major 4/4 torch.Size([128, 130])
English G major 4/4 torch.Size([266, 130])
English A major 3/2 torch.Size([144, 130])
English D major 4/4 torch.Size([256, 130])
English e minor 4/4 torch.Size([384, 130])
English D major 4/4 torch.Size([192, 130])
English A major 4/4 torch.Size([384, 130])
English C major 3/4 torch.Size([760, 130])
English C major 4/4 torch.Size([256, 130])
English A major 4/4 torch.Size([126, 130])
English D major 4/4 torch.Size([224, 130])
English D major 4/4 torch.Size([712, 130])
English B- major 3/8 torch.Size([138, 130])
English G major 3/4 torch.Size([552, 130])
English A major 4/4 torch.Size([127, 130])
English D major 4/4 torch.Size([136, 130])
English D 

English G major 3/4 torch.Size([200, 130])
English D major 4/4 torch.Size([256, 130])
English D major 4/4 torch.Size([252, 130])
English G major 3/8 torch.Size([192, 130])
English B- major 4/4 torch.Size([184, 130])
English C major 3/2 torch.Size([288, 130])
English G major 4/4 torch.Size([364, 130])
English C major 3/8 torch.Size([147, 130])
English A major 4/4 torch.Size([132, 130])
English C major 4/4 torch.Size([256, 130])
English B- major 3/2 torch.Size([1056, 130])
English G major 4/4 torch.Size([196, 130])
English G major 4/4 torch.Size([326, 130])
English C major 4/4 torch.Size([192, 130])
English G major 3/4 torch.Size([192, 130])
English F major 4/4 torch.Size([191, 130])
English C major 3/4 torch.Size([192, 130])
English F major 4/4 torch.Size([258, 130])
English G major 4/4 torch.Size([188, 130])
English B- major 4/4 torch.Size([128, 130])
English D major 4/4 torch.Size([264, 130])
English D major 4/4 torch.Size([256, 130])
English D major 3/8 torch.Size([288, 130])
English

English G major 3/8 torch.Size([144, 130])
English D major 4/4 torch.Size([182, 130])
English F major 4/4 torch.Size([256, 130])
English A major 3/8 torch.Size([96, 130])
English G major 3/4 torch.Size([192, 130])
English D major 4/4 torch.Size([194, 130])
English D major 4/4 torch.Size([128, 130])
English D major 3/8 torch.Size([144, 130])
English B- major 3/2 torch.Size([474, 130])
English G major 4/4 torch.Size([64, 130])
English D major 3/4 torch.Size([168, 130])
English G major 4/4 torch.Size([239, 130])
English D major 4/4 torch.Size([258, 130])
English G major 4/4 torch.Size([250, 130])
English D major 4/4 torch.Size([588, 130])
English D major 3/8 torch.Size([286, 130])
English G major 4/4 torch.Size([255, 130])
English G major 3/2 torch.Size([168, 130])
English G major 4/4 torch.Size([192, 130])
English F major 3/2 torch.Size([288, 130])
English G major 3/8 torch.Size([136, 130])
English G major 3/8 torch.Size([96, 130])
English G major 4/4 torch.Size([130, 130])
English D maj

In [7]:
with open("/gpfsnyu/scratch/yz6492/multimodal/data/MusicData_Ca.pkl", 'rb') as f:
    counter = 0
    filter_list = list()
    print('in')
    data = pickle.load(f)
    key_stat = [i.key for i in data]
    dict = {}
    for key in key_stat:
        dict[key] = dict.get(key, 0) + 1
    print(dict)
    print(counter)

in
{'C major': 6711, 'a minor': 1285}
0


In [10]:
# clean
with open("/gpfsnyu/scratch/yz6492/multimodal/data/MusicData_full.pkl", 'rb') as f:
    counter = 0
    irish_counter = 0
    filter_list = list()
    print('in')
    data = pickle.load(f)
    for line in data:
        if line.meter in ['4/4','2/4','3/4','6/8','3/8'] and line.key in ['C major','a minor']:# and line.culture != 'English':
            if line.culture == 'Irish':
                irish_counter += 1
                if irish_counter > 5000:
                    continue
            filter_list.append(line)
            print(line.culture, line.key, line.meter)
            counter += 1
    with open("/gpfsnyu/scratch/yz6492/multimodal/data/MusicData_Ca_new.pkl", 'wb') as g:
        pickle.dump(filter_list, g)
    print(counter)

in
Irish C major 4/4
Irish C major 4/4
Irish C major 6/8
Irish C major 6/8
Irish C major 4/4
Irish a minor 3/4
Irish C major 4/4
Irish C major 6/8
Irish C major 3/4
Irish C major 4/4
Irish C major 4/4
Irish C major 4/4
Irish a minor 6/8
Irish C major 4/4
Irish C major 2/4
Irish C major 4/4
Irish C major 4/4
Irish C major 6/8
Irish C major 4/4
Irish a minor 3/4
Irish C major 4/4
Irish C major 3/4
Irish C major 4/4
Irish a minor 4/4
Irish a minor 3/4
Irish a minor 3/4
Irish C major 4/4
Irish C major 4/4
Irish a minor 4/4
Irish C major 4/4
Irish a minor 4/4
Irish a minor 3/4
Irish C major 3/4
Irish C major 6/8
Irish C major 4/4
Irish C major 6/8
Irish C major 4/4
Irish C major 4/4
Irish C major 6/8
Irish C major 4/4
Irish C major 4/4
Irish C major 6/8
Irish C major 6/8
Irish C major 4/4
Irish C major 4/4
Irish C major 3/4
Irish a minor 4/4
Irish C major 4/4
Irish C major 6/8
Irish C major 4/4
Irish a minor 3/4
Irish C major 4/4
Irish C major 4/4
Irish a minor 4/4
Irish C major 4/4
Irish a

Irish a minor 4/4
Irish C major 4/4
Irish C major 4/4
Irish a minor 6/8
Irish C major 4/4
Irish C major 4/4
Irish C major 4/4
Irish C major 4/4
Irish C major 6/8
Irish C major 4/4
Irish C major 6/8
Irish C major 3/4
Irish C major 4/4
Irish C major 4/4
Irish C major 4/4
Irish C major 4/4
Irish C major 4/4
Irish C major 4/4
Irish C major 3/4
Irish C major 3/4
Irish a minor 4/4
Irish C major 4/4
Irish C major 6/8
Irish C major 4/4
Irish C major 6/8
Irish C major 4/4
Irish C major 3/4
Irish C major 4/4
Irish C major 4/4
Irish C major 4/4
Irish a minor 4/4
Irish C major 4/4
Irish C major 6/8
Irish C major 2/4
Irish C major 6/8
Irish C major 4/4
Irish C major 3/4
Irish C major 4/4
Irish C major 4/4
Irish C major 4/4
Irish C major 4/4
Irish C major 6/8
Irish C major 4/4
Irish C major 4/4
Irish C major 6/8
Irish C major 6/8
Irish C major 6/8
Irish C major 4/4
Irish C major 6/8
Irish a minor 6/8
Irish a minor 6/8
Irish C major 4/4
Irish C major 4/4
Irish C major 3/4
Irish C major 3/4
Irish C ma

Irish C major 4/4
Irish a minor 4/4
Irish C major 4/4
Irish C major 4/4
Irish C major 4/4
Irish C major 4/4
Irish C major 4/4
Irish C major 6/8
Irish C major 4/4
Irish C major 3/4
Irish a minor 2/4
Irish C major 6/8
Irish a minor 4/4
Irish C major 6/8
Irish C major 4/4
Irish C major 4/4
Irish C major 4/4
Irish C major 4/4
Irish a minor 6/8
Irish C major 6/8
Irish C major 6/8
Irish C major 6/8
Irish C major 4/4
Irish a minor 4/4
Irish C major 4/4
Irish C major 3/4
Irish C major 6/8
Irish C major 3/4
Irish C major 6/8
Irish C major 3/4
Irish C major 4/4
Irish C major 4/4
Irish C major 6/8
Irish C major 4/4
Irish C major 6/8
Irish C major 6/8
Irish C major 6/8
Irish C major 6/8
Irish C major 6/8
Irish a minor 3/4
Irish C major 6/8
Irish C major 4/4
Irish C major 4/4
Irish a minor 4/4
Irish a minor 6/8
Irish C major 4/4
Irish C major 6/8
Irish C major 2/4
Irish C major 3/4
Irish C major 4/4
Irish C major 4/4
Irish C major 6/8
Irish C major 4/4
Irish C major 4/4
Irish C major 3/4
Irish a mi

Irish C major 6/8
Irish C major 4/4
Irish a minor 3/4
Irish C major 4/4
Irish C major 6/8
Irish C major 4/4
Irish C major 3/4
Irish C major 6/8
Irish C major 4/4
Irish C major 4/4
Irish a minor 6/8
Irish C major 6/8
Irish C major 4/4
Irish C major 4/4
Irish C major 4/4
Irish C major 6/8
Irish C major 4/4
Irish C major 6/8
Irish a minor 3/4
Irish C major 4/4
Irish C major 4/4
Irish C major 4/4
Irish a minor 4/4
Irish C major 4/4
Irish C major 6/8
Irish C major 4/4
Irish C major 3/4
Irish C major 4/4
Irish a minor 6/8
Irish C major 4/4
Irish a minor 3/8
Irish C major 4/4
Irish C major 4/4
Irish C major 4/4
Irish C major 4/4
Irish C major 3/4
Irish a minor 4/4
Irish C major 4/4
Irish C major 4/4
Irish C major 4/4
Irish C major 4/4
Irish C major 4/4
Irish a minor 4/4
Irish C major 6/8
Irish C major 4/4
Irish C major 4/4
Irish C major 4/4
Irish C major 6/8
Irish C major 4/4
Irish C major 4/4
Irish C major 6/8
Irish C major 4/4
Irish C major 4/4
Irish a minor 2/4
Irish C major 4/4
Irish C ma

Irish C major 6/8
Irish C major 6/8
Irish C major 3/4
Irish C major 4/4
Irish C major 4/4
Irish C major 6/8
Irish C major 4/4
Irish C major 6/8
Irish C major 6/8
Irish C major 6/8
Irish C major 4/4
Irish C major 4/4
Irish C major 4/4
Irish C major 6/8
Irish a minor 4/4
Irish C major 4/4
Irish C major 4/4
Irish C major 6/8
Irish a minor 6/8
Irish C major 4/4
Irish C major 4/4
Irish C major 6/8
Irish a minor 4/4
Irish C major 4/4
Irish C major 6/8
Irish a minor 6/8
Irish C major 6/8
Irish C major 4/4
Irish C major 6/8
Irish a minor 3/4
Irish C major 6/8
Irish C major 4/4
Irish C major 4/4
Irish C major 4/4
Irish C major 4/4
Irish C major 4/4
Irish C major 4/4
Irish C major 4/4
Irish a minor 4/4
Irish a minor 6/8
Irish C major 4/4
Irish C major 3/4
Irish C major 4/4
Irish C major 6/8
Irish C major 6/8
Irish C major 4/4
Irish C major 4/4
Irish C major 6/8
Irish C major 4/4
Irish C major 4/4
Irish C major 3/4
Irish C major 4/4
Irish C major 6/8
Irish C major 2/4
Irish a minor 4/4
Irish C ma

Chinese C major 4/4
Chinese C major 2/4
Chinese C major 2/4
Chinese C major 2/4
Chinese C major 3/4
Chinese C major 4/4
Chinese C major 2/4
Chinese C major 2/4
Chinese C major 2/4
Chinese C major 4/4
Chinese C major 2/4
Chinese C major 2/4
Chinese C major 2/4
Chinese C major 2/4
Chinese C major 2/4
Chinese C major 2/4
Chinese C major 2/4
Chinese C major 2/4
Chinese C major 2/4
Chinese C major 4/4
Chinese C major 2/4
Chinese C major 2/4
Chinese C major 2/4
Chinese C major 2/4
Chinese C major 4/4
Chinese C major 2/4
Chinese C major 2/4
Chinese C major 2/4
Chinese C major 2/4
Chinese C major 2/4
Chinese C major 2/4
Chinese C major 2/4
Chinese C major 2/4
Chinese C major 2/4
Chinese C major 2/4
Chinese C major 4/4
Chinese C major 2/4
Chinese C major 2/4
Chinese C major 2/4
English C major 3/4
English C major 4/4
English C major 6/8
English C major 6/8
English C major 6/8
English C major 2/4
English C major 6/8
English C major 3/4
English C major 4/4
English C major 6/8
English C major 6/8


5701
